In [28]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np

from tensorflow.keras.layers import Dense, SimpleRNN, Input, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.utils import to_categorical

In [29]:

with open('krovostok.txt', 'r', encoding='utf-8') as f:
    texts = f.read()
    texts = texts.replace('\ufeff', '')  # убираем первый невидимый символ

In [30]:
maxWordsCount = 2000 # максимум слов - maxWordsCount - 1 (пробел) 
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!"#$%&amp;()*+-/:;<=>@[\\]^_`{|}~\t\n\r«»',
                      lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts([texts])

In [31]:
dist = list(tokenizer.word_counts.items()) # посмотреть частоту встречаемости в тексте
print(dist[:20])

[('думай', 2), ('позитивно,', 2), ('стакан', 2), ('всегда', 7), ('наполовину', 2), ('полон,', 2), ('чувствуй', 2), ('хорошее,', 2), ('плохого', 3), ('не', 104), ('существует,', 2), ('между', 3), ('нет', 4), ('и', 112), ('да', 16), ('выбор', 3), ('только', 12), ('верь', 2), ('в', 133), ('лучшее,', 2)]


In [41]:
data = tokenizer.texts_to_sequences([texts]) # преобразуем текст в последовательность чисел в соотв-и с полученный словарем
#res = to_categorical(data[0], num_classes=maxWordsCount) #преобразование в one hot векторы
#print(res.shape)

res = np.array(data[0])

In [46]:
inp_words = 2 # кол-во слов, на основе которых строится предсказание
n = res.shape[0] - inp_words

#X = np.array([res[i:i + inp_words, :] for i in range(n)])
X = np.array([res[i:i + inp_words] for i in range(n)])
#X = np.array([res[i:i + inp_words] for i in range(n)])
Y = to_categorical(res[inp_words:], num_classes=maxWordsCount) # должен быть не набором чисел, а набором векторов ?
#Y = res[inp_words:] # тензор слов, которые мы должны спрогнозировать


In [47]:
model = Sequential()
#model.add(Input((inp_words, maxWordsCount)))
model.add(Embedding(maxWordsCount, 256, input_length = inp_words))
model.add(SimpleRNN(128, activation='tanh'))
model.add(Dense(maxWordsCount, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 2, 256)            512000    
                                                                 
 simple_rnn_5 (SimpleRNN)    (None, 128)               49280     
                                                                 
 dense_5 (Dense)             (None, 2000)              258000    
                                                                 
Total params: 819,280
Trainable params: 819,280
Non-trainable params: 0
_________________________________________________________________


In [48]:
history = model.fit(X, Y, batch_size=32, epochs=50)

Epoch 1/50
125/125 [==============================] - 2s 10ms/step - loss: 7.4849 - accuracy: 0.0259
Epoch 2/50
125/125 [==============================] - 1s 8ms/step - loss: 6.6712 - accuracy: 0.0450
Epoch 3/50
125/125 [==============================] - 1s 8ms/step - loss: 6.0912 - accuracy: 0.0651
Epoch 4/50
125/125 [==============================] - 1s 9ms/step - loss: 5.3959 - accuracy: 0.1146
Epoch 5/50
125/125 [==============================] - 1s 8ms/step - loss: 4.5657 - accuracy: 0.2166
Epoch 6/50
125/125 [==============================] - 1s 8ms/step - loss: 3.7238 - accuracy: 0.3720
Epoch 7/50
125/125 [==============================] - 1s 8ms/step - loss: 2.9801 - accuracy: 0.5089
Epoch 8/50
125/125 [==============================] - 1s 10ms/step - loss: 2.3517 - accuracy: 0.6268
Epoch 9/50
125/125 [==============================] - 2s 13ms/step - loss: 1.8415 - accuracy: 0.7024
Epoch 10/50
125/125 [==============================] - 2s 14ms/step - loss: 1.4436 - accuracy: 0.

In [49]:
def buildPhrase(texts, str_len=20):
    res = texts
    data = tokenizer.texts_to_sequences([texts])[0]
    for i in range(str_len):
        #x = to_categorical(data[i: i + inp_words], num_classes=maxWordsCount)  # преобразуем в One-Hot-encoding
        #inp = x.reshape(1, inp_words, maxWordsCount)
        x = data[i: i + inp_words]
        inp = np.expand_dims(x, axis=0)

        pred = model.predict(inp)
        indx = pred.argmax(axis=1)[0]
        data.append(indx)

        res += " " + tokenizer.index_word[indx]  # дописываем строку ( преобразовываем индекс обратно в слово) 

    return res


In [59]:
res = buildPhrase("говно моча")
print(res)

говно моча ебать ударила в голову в 4 активно ругался матом в детском саду девочки впервые показали мне пизду потом школа, вонючая
